- [문제 링크](https://www.acmicpc.net/problem/11780)

In [77]:
import sys
sys.stdin = open('test.txt', 'r')
input = sys.stdin.readline

n = int(input())
m = int(input())
INF = int(1e9)
# INF = float('inf')

graph = [[INF] * (n + 1) for _ in range(n + 1)]
pre_index = [[-1] * (n + 1) for i in range(n + 1)] # 점 j에서 출발해 k까지 갈 때, k에 오기 직전의 인덱스 값

for _ in range(m):
    a, b, w = map(int, input().split())
    pre_index[a][b] = a # 간선이 존재하는 점에만 row로 초기화해줌. 간선이 없다면 -1로 유지됨

    if graph[a][b] > w:
        graph[a][b] = w
        # a에서 b로 가는 경로에서, 마지막 점의 이전 점은 기본적으로 a임

def fw():
    for i in range(1, n+1):
        graph[i][i] = 0
        pre_index[i][i] = -1 
        for j in range(1, n+1):
            for k in range(1, n+1):
                if graph[j][k] > graph[j][i] + graph[i][k]:
                    graph[j][k] = graph[j][i] + graph[i][k]
                    pre_index[j][k] = pre_index[i][k]
                    # pre_index[j][k] = i # 이거는 성립이 안됨
                    
fw()
# 1. 도시 i->j로 가는데 필요한 최소 비용 출력
for i in graph[1:]:
    for j in i[1:]:
        print(0 if j == INF else j, end = " ")
    print()

# 2. 
for j in range(1, n + 1):
    for k in range(1, n + 1):
        if pre_index[j][k] == -1:
            print(0)
            continue
        
        ans = [k]
        while True:
            ans.append(pre_index[j][k])
            k = pre_index[j][k]
            if k == j:
                break
                
            
            
            
        print(len(ans), *ans[::-1])

0 2 3 1 4 
12 0 15 2 5 
8 5 0 1 1 
10 7 13 0 3 
7 4 10 6 0 
0
2 1 2
2 1 3
2 1 4
3 1 3 5
4 2 4 5 1
0
5 2 4 5 1 3
2 2 4
3 2 4 5
2 3 1
3 3 5 2
0
2 3 4
2 3 5
3 4 5 1
3 4 5 2
4 4 5 1 3
0
2 4 5
2 5 1
2 5 2
3 5 1 3
3 5 2 4
0


### 플로이드 워셜이 뭐였드라?
- 그래프의 모든 정점에 대해 최소 경로를 구하는 알고리즘
- $O(N^3)$
1. 그래프 : 경로가 없더라도 모든 케이스를 그림. 자기 자신은 0
2. 그렇게 하면 들어온 간선들에 대해 `graph[a][b] = w`의 식을 만들 수 있음 : 이미 그래프에 있는 값이 w보다 크다면, w로 갱신함
3. 중간노드 1번부터 반복문을 돌림(`i`)
    - 모든 시작점(`j`)과 끝점(`k`)에 대해, `graph[j][k]`를 `i`를 경유한 `graph[j][i] + graph[i][k]` 값과 기존 값중 더 작은 값으로 갱신해나감
    
### 역추적
- 경로 행렬을 거리 행렬과 같은 크기로 만든다
- 이 경로 행렬의 각 칸의 의미는 `row -> col` 이동 시 경유하는 마지막 노드가 됨
- 따라서 간선이 주어진다면, 초기화는 `row` 인덱스로 해준다 : 간선이 있으니까 경로가 있다는 의미이고, 크기야 어찌됐든 초기화가 가능한 상황이므로.
- 그러면 어떻게 갱신할까?
- 플로이드-워셜 알고리즘은 모든 노드를 중간 노드로 놓은 다음, 각 시작점과 끝점에 대해 중간 노드를 지날 때의 크기와 기존 크기 간 비교해서 작은 값으로 갱신한다고 했음
- 추적하는 방법 자체는 **끝 노드에서 이전 노드를 가리키는 인덱스를 저장하는 배열**이 있으면 될 거임
    - 최솟값을 갱신하는 과정에서 인덱스를 저장한다면, 이 인덱스를 갖는 노드 자체가 최솟값 경로의 일원이 되는 거임
- 이를 기록하기 위해, 다시 플로이드 - 워셜 알고리즘을 생각해보면..
> 중간점(`i`)이 1번 노드부터 마지막 노드까지 돌아감 (시작점 `j`, 도착점 `k`)   
> 이걸 수식으로 보면 `graph[j][k]`와 `graph[j][i] + graph[i][k]` 값을 비교하게 될 거임  
> 갱신할 수 있는 상황(후자의 값이 더 작다면)이라면, 나중에 어찌됐든 간에 현재 `j`에서 `k`로 가는 경로는 `i`를 반드시 경유하게 되어 있음(최소 거리니까)  
> 따라서 **현재 시점에서 `k`로 갈 수 있는 가장 직전 노드를 `i`라고 할 수 있다는 거**임
> 근데 한 발 더 나아가야 함 - `i`는 1부터 순차적으로 돌기 때문에 놓치는 케이스들이 발생함 (내 경우 `[2][3]`에서 경로가 `2 4 5 3`으로 떴음 : 정답은 `2 4 5 1 3`(`5->3` 경로가 없음))  
> `j->i->k`로 가는 경로와, `i->k`로 가는 경로를 생각해보면, `경로행렬[j][k]`와 `경로행렬[i][k]` 값은 동일할 거임 : 전자가 후자를 포함하기 때문  
> 따라서 경로 행렬에 저장하는 이전 노드는 `경로행렬[j][k] = 경로행렬[i][k]`가 된다. (값이 없는 경우는 당연히 걱정 안해도 됨. 최솟값이 갱신되는 상황은 경로 행렬에 있는 값이 `없음 표시`가 아니기 때문)
